In [ ]:
import pandas as pd
import os
import geopandas as gpd
import numpy as np
import fiona
from shapely.geometry import shape
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from shapely.geometry import box


### Load data

In [ ]:
dir_path = '../data/external/water_stations_data/water_temp'
#dir_path = '../../tfg/data/15994_4699_2024-10-31_11-54-16'
stations_meta = pd.read_excel('../data/external/Stationsliste_HydrowebSAM_EN_july2023.xlsx',skiprows=range(0,4))
stations_meta.rename(columns={'Station number': 'Stationsnummer'}, inplace=True)
stations_meta.drop_duplicates(subset = ['Stationsnummer'], inplace=True)

### Obtain georeference about stations that have data

In [ ]:
stations_ds={}
new_df = pd.DataFrame()
for file in os.listdir(dir_path):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(dir_path, file), delimiter=';', encoding='latin1', skiprows=range(0,8))
        st_number = df.Stationsnummer.unique()
        st_number_str = st_number[0]
        
        if len(st_number) > 1:
            i = 1
            while st_number_str == 'Stationsnummer' or np.isnan(float(st_number_str)):
                st_number_str = st_number[i]
                i+=1
            print('More than one station number')
        print(st_number_str)
        if st_number_str in stations_meta['Stationsnummer'].values:
            print('Station number found')
            new_df = pd.concat([new_df,stations_meta[stations_meta['Stationsnummer']==st_number_str]])
            
        d = "2013-03-01"
        #df["Zeitstempel"] = pd.to_datetime(df["Zeitstempel"])
        df = df[df.Zeitstempel >= d]
        stations_ds[st_number_str] = df
            

### Obtain grid cells sorrounding every station

In [ ]:
crs_original = "EPSG:2056"  # o usa "EPSG:21781" si las coordenadas están en LV03

# Convertir el DataFrame de pandas a un GeoDataFrame de geopandas
gdf = gpd.GeoDataFrame(
    new_df,
    geometry=gpd.points_from_xy(new_df["X-Coordinate"], new_df["Y-Coordinate"]),
    crs=crs_original
)

# Proyectar al CRS deseado, por ejemplo WGS84 (EPSG:4326)
gdf = gdf.to_crs("EPSG:4326")


In [ ]:
gdf.plot()

In [ ]:
import contextily as ctx
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color="blue", markersize=5, alpha=0.7, label="Puntos")

# Añadir un mapa base
ctx.add_basemap(ax, crs="EPSG:4326", source=ctx.providers.OpenStreetMap.Mapnik)
# Configurar título y etiquetas
ax.set_title("Visualización de Coordenadas Geográficas", fontsize=15)
ax.set_xlabel("Longitud", fontsize=12)
ax.set_ylabel("Latitud", fontsize=12)
plt.legend()
plt.show()

#### Create a shp for each cell

In [ ]:
water_ds = pd.DataFrame(index = stations_ds[2161].Zeitstempel,columns = range(1,len(gdf)+1))

In [ ]:
# Change the CRS to a projected coordinate system (e.g., UTM appropriate for your region)
gdf = gdf.to_crs("EPSG:32633")  # Make sure to use the correct CRS for your region

# Define the buffer size in meters (10x10 km = 5000 m in each direction from the center point)
buffer_size = 5000

# Create a folder to store the shapefiles if it doesn't exist
output_folder = "../data/external/shp/cells_10x10km"
os.makedirs(output_folder, exist_ok=True)

cell_data = pd.DataFrame(columns=["Cell", "Latitude", "Longitude"])

c=1
# Iterate over each point and create a 10x10 km cell, saving it as an individual shapefile
for idx, point in gdf.iterrows():
    # Create the square cell
    x, y = point.geometry.x, point.geometry.y
    square = box(x - buffer_size, y - buffer_size, x + buffer_size, y + buffer_size)
    
    # Create a new GeoDataFrame for the cell
    cell_gdf = gpd.GeoDataFrame(index=[0], crs=gdf.crs, geometry=[square])
    
    # Define the filename for each cell, using the index or any relevant attribute
    cell_filename = f"{output_folder}/cell_{c}.shp"
    cell_gdf = cell_gdf.to_crs("EPSG:2056")
    
    # Save the individual cell as a shapefile
    cell_gdf.to_file(cell_filename, driver="ESRI Shapefile")
    try:
        station_number = point.Stationsnummer
        station_data = stations_ds[station_number]
        water_ds[c] = list(station_data.Wert.values)

        latitude = point['X-Coordinate']  # Taking the first value if multiple exist
        longitude = point['Y-Coordinate']  # Taking the first value if multiple exist

        # Append data to cell_data DataFrame
        cell_data = pd.concat([cell_data,pd.DataFrame({"Cell": [c], "Latitude": [latitude], "Longitude": [longitude]})], ignore_index=True)
    
    except Exception as e:
        print(e)
        print(station_number,c, len(list(station_data.Wert.values)))
    
    
    c+=1
    

print("Individual shapefiles created in the 'cells_10x10km' folder.")


In [ ]:
pd.read_csv(f'{dirr}/cell_coordinates.csv')#, index_col=0)

In [ ]:
dirr='../data/raw/wt'
os.makedirs(dirr,exist_ok=True)
water_ds.to_csv(f'{dirr}/water_temp.csv',index=True)
cell_data.to_csv(f'{dirr}/cell_coordinates.csv',index=False)
pd.read_csv(f'{dirr}/water_temp.csv', index_col=0)

In [ ]:
a=gpd.read_file(os.path.join(output_folder,'cell_1.shp'))
a.plot()
a.crs

#### Plot all cells

In [ ]:
gdf = gdf.to_crs("EPSG:32633")
buffer_size = 5000  # 10x10 km significa 5 km de radio en cada dirección desde el centro

# Función para crear una celda cuadrada alrededor de cada punto
def create_square(point, size):
    x, y = point.x, point.y
    return box(x - size, y - size, x + size, y + size)

# Aplicamos la función a cada punto en el GeoDataFrame
gdf['geometry'] = gdf.geometry.apply(lambda point: create_square(point, buffer_size))
gdf = gdf.to_crs("EPSG:2056")
gdf.plot(color='none')

### Comparison to old cells

In [ ]:
crs_original = "EPSG:2056"  # o usa "EPSG:21781" si las coordenadas están en LV03

# Convertir el DataFrame de pandas a un GeoDataFrame de geopandas
gdf = gpd.GeoDataFrame(
    new_df,
    geometry=gpd.points_from_xy(new_df["X-Coordinate"], new_df["Y-Coordinate"]),
    crs=crs_original
)

# Proyectar al CRS deseado, por ejemplo WGS84 (EPSG:4326)
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
gdf2 = gpd.read_file('../data/external/shp/Switzerland_shapefile')
gdf2 = gdf2.to_crs("EPSG:4326")
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf2.plot(ax=ax, facecolor="none", edgecolor="green", linewidth=1, label='Preprocessed Cells')
gdf.plot(ax=ax)
